die sql befehle um tx und wei auszulesen scheinen zu funktionieren.
den wei befehl habe ich zusötzlich zum sample datensatz mit dem balance befehl im inet überprüft. 

Todo: die 4 eigenschaften für die 40 tausend adressen raussuchen.

In [154]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.cloud import bigquery
import time
%env GOOGLE_APPLICATION_CREDENTIALS=../bigquery-service-account.json 
client = bigquery.Client()

env: GOOGLE_APPLICATION_CREDENTIALS=../bigquery-service-account.json


In [155]:
data = []
for index, column in schema.iterrows():
    data.append((column["column_name"], column["data_type"]))
s0 = [" ".join(d) for d in data]
s1 = ",".join(s0)

In [156]:
query = """
create or replace table `masterarbeit-245718.ethereum_us.test_tempTable` (
{}
);
""".format(s1);

print(query);

client.query(query).result();


create or replace table `masterarbeit-245718.ethereum_us.test_tempTable` (
transaction_hash STRING,transaction_index INT64,from_address STRING,to_address STRING,value NUMERIC,input STRING,output STRING,trace_type STRING,call_type STRING,reward_type STRING,gas INT64,gas_used INT64,subtraces INT64,trace_address STRING,error STRING,status INT64,block_timestamp TIMESTAMP,block_number INT64,block_hash STRING
);



In [157]:
def send(_from ,_to, _value):
    query = """
    INSERT INTO `masterarbeit-245718.ethereum_us.test_tempTable` VALUES (
    "transaction_hash",
    1,
    "{}",
    "{}",
    {},
    "input",
    "output",
    "trace_type",
    "call",
    "reward_type",
    1,
    1,
    1,
    "trace_address",
    "error",
    1,
    "0001-01-01 00:00:00",
    1,
    "block_hash");
    """.format(_from, _to, _value)
    return client.query(query)

In [158]:
for i in range(0,2,1):
    queryJob = send("0x0","0x1",2)

queryJob.result();

In [159]:
for i in range(0,3,1):
    queryJob = send("0x1","0x0",1)

queryJob.result();

In [160]:
query = "select * from `masterarbeit-245718.ethereum_us.test_tempTable`;"
data = client.query(query).result().to_dataframe(); 
data

,transaction_hash,transaction_index,from_address,to_address,value,input,output,trace_type,call_type,reward_type,gas,gas_used,subtraces,trace_address,error,status,block_timestamp,block_number,block_hash
0,transaction_hash,1,0x1,0x0,1,input,output,trace_type,call,reward_type,1,1,1,trace_address,error,1,0001-01-01 00:00:00+00:00,1,block_hash
1,transaction_hash,1,0x1,0x0,1,input,output,trace_type,call,reward_type,1,1,1,trace_address,error,1,0001-01-01 00:00:00+00:00,1,block_hash
2,transaction_hash,1,0x0,0x1,2,input,output,trace_type,call,reward_type,1,1,1,trace_address,error,1,0001-01-01 00:00:00+00:00,1,block_hash
3,transaction_hash,1,0x1,0x0,1,input,output,trace_type,call,reward_type,1,1,1,trace_address,error,1,0001-01-01 00:00:00+00:00,1,block_hash
4,transaction_hash,1,0x0,0x1,2,input,output,trace_type,call,reward_type,1,1,1,trace_address,error,1,0001-01-01 00:00:00+00:00,1,block_hash


In [161]:
query = """
  with weiReceivedView as (
    -- debits
    select to_address, sum(ifnull(value, 0)) as weiReceived
    from `masterarbeit-245718.ethereum_us.test_tempTable`
    where to_address is not null
    and status = 1
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    group by to_address
), weiSentView as (
    -- credits
    select from_address, sum(ifnull(value, 0)) as weiSent
    from `masterarbeit-245718.ethereum_us.test_tempTable`
    where from_address is not null
    and status = 1
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    group by from_address
) select 
CASE 
  when to_address is not null then to_address
  when from_address is not null then from_address
end as address, ifnull(weiReceived,0) as weiReceived, ifnull(weiSent,0) as weiSent
from weiReceivedView full outer join weiSentView on from_address = to_address
"""

data = client.query(query).result().to_dataframe();
data

,address,weiReceived,weiSent
0,0x0,3,4
1,0x1,4,3


In [162]:
query = """
with txSent as (
  SELECT from_address, count(*) as numberOfTranscationsSent FROM `masterarbeit-245718.ethereum_us.test_tempTable` 
  where to_address is not null and status = 1 and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null) 
  group by from_address
), txReceived as (
  SELECT to_address, count(*) as numberOfTranscationsReceived FROM `masterarbeit-245718.ethereum_us.test_tempTable` 
  where to_address is not null and status = 1 and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
  group by to_address
) SELECT 
CASE  
  WHEN to_address IS NOT NULL THEN to_address
  WHEN from_address IS NOT NULL THEN from_address
END AS address,
IFNULL(numberOfTranscationsReceived, 0) as numberOfTranscationsReceived, 
IFNULL(numberOfTranscationsSent, 0) as numberOfTranscationsSent
from txReceived FULL OUTER JOIN txSent on to_address = from_address
"""

data = client.query(query).result().to_dataframe();
data

,address,numberOfTranscationsReceived,numberOfTranscationsSent
0,0x1,2,3
1,0x0,3,2


In [163]:
query = """
  with weiReceivedView as (
    -- debits
    select to_address, sum(ifnull(value, 0)) as weiReceived
    from `masterarbeit-245718.ethereum_us.top40k_week1777_traces`
    where to_address is not null
    and status = 1
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    group by to_address
), weiSentView as (
    -- credits
    select from_address, sum(ifnull(value, 0)) as weiSent
    from `masterarbeit-245718.ethereum_us.top40k_week1777_traces`
    where from_address is not null
    and status = 1
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    group by from_address
) select 
CASE 
  when to_address is not null then to_address
  when from_address is not null then from_address
end as address, ifnull(weiReceived,0) as weiReceived, ifnull(weiSent,0) as weiSent
from weiReceivedView full outer join weiSentView on from_address = to_address
"""

data1 = client.query(query).result().to_dataframe();
data1

,address,weiReceived,weiSent
0,0xe07092b75db3f19a3bb1eca29ef3470d2875987b,0,500000000000000000
1,0x212b32f8ab8a1b5751632ba0729149c5db98348a,302993453549880289131,266377845750000000000
2,0x048ed42e9b7481ae59dcd52ea346c6cf9dd640a0,1239796414000000000000,1300000000000000000000
3,0x166c0696941e10b61605a989e3558849654ed52d,1435899530000000000,1435899530000000000
4,0x7a3370075a54b187d7bd5dcebf0ff2b5552d4f7d,2579170802230166007866,2210228415510999139707
5,0x62108dbdb0f848d703797a025b391aecc9c373e9,102095191254866485,102095200000000000
6,0x76246093a74649178d309dfc6a647be96c6d5b72,9260791200000000000,9259657199999999999
7,0x59ac000619b5dad477f575165cc6c8bec0fce9db,1306082200000000000,3835242200000000000
8,0x9df0e855c1dc9cddc17e80698719163afff8574c,6820250500000000000,6816281500000000000
9,0x621a7761b399fa5b6d333e636726378d6dae6c1c,3817000000000000000,3820486887000000000


In [142]:
data1["balance"] = data1["weiReceived"] - data1["weiSent"]
data1.sort_values(by="balance", ascending=False)

,address,weiReceived,weiSent,balance
228121,0xdec87ed2dcdf63043917df0712a5f8866e12cc24,104351588165584364427028,1000000000000000000,104350588165584364427028
533174,0x4e9ce36e442e55ecd9025b9a6e0d88485d628a67,55744745076118533591785,0,55744745076118533591785
103704,0x6cc5f688a315f3dc28a7781717a9a798a59fda7b,379299459580936873870052,323899884000000000000000,55399575580936873870052
130909,0xeba290cf248cb14442a071fbcb58a9cc5dcde28e,188908832092361773104756,136944446000000000000000,51964386092361773104756
200586,0xd8a83b72377476d0a66683cde20a8aad0b628713,110673194938050000000000,67966000000000000000000,42707194938050000000000
458134,0xc1de11585557c6b26272759cb148170d832ab7a2,35210000000000000000000,0,35210000000000000000000
228120,0x5861b8446a2f6e19a067874c133f04c578928727,121169959201827000000000,87956000000000000000000,33213959201827000000000
27352,0x4ddc2d193948926d02f9b1fe9e1daa0718270ed5,28682495977094995552645,4401177963719257074406,24281318013375738478239
167314,0x6cb2b01fc4ee005e86a4acefd8ae19d06e0688ce,25500000000000000000000,1685719000000000000000,23814281000000000000000
246164,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,121453233624411185423913,98430249125171519641481,23022984499239665782432


In [143]:
query = """
with double_entry_book as (
    -- debits
    select to_address as address, value as value
    from `masterarbeit-245718.ethereum_us.top40k_week1777_traces`
    where to_address is not null
    and status = 1
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    union all
    -- credits
    select from_address as address, -value as value
    from `masterarbeit-245718.ethereum_us.top40k_week1777_traces`
    where from_address is not null
    and status = 1
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
)
select address, sum(value) as balance
from double_entry_book
group by address
order by balance desc
"""

data2 = client.query(query).result().to_dataframe();
data2

,address,balance
0,0xdec87ed2dcdf63043917df0712a5f8866e12cc24,104350588165584364427028
1,0x4e9ce36e442e55ecd9025b9a6e0d88485d628a67,55744745076118533591785
2,0x6cc5f688a315f3dc28a7781717a9a798a59fda7b,55399575580936873870052
3,0xeba290cf248cb14442a071fbcb58a9cc5dcde28e,51964386092361773104756
4,0xd8a83b72377476d0a66683cde20a8aad0b628713,42707194938050000000000
5,0xc1de11585557c6b26272759cb148170d832ab7a2,35210000000000000000000
6,0x5861b8446a2f6e19a067874c133f04c578928727,33213959201827000000000
7,0x4ddc2d193948926d02f9b1fe9e1daa0718270ed5,24281318013375738478239
8,0x6cb2b01fc4ee005e86a4acefd8ae19d06e0688ce,23814281000000000000000
9,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,23022984499239665782432
